In [ ]:
import aisdb
from aisdb import DBQuery
from aisdb.database.dbconn import PostgresDBConn
from datetime import datetime
from aisdb.weather.weather_fetch import ClimateDataStore 

In [ ]:
# >>> PostgreSQL Information <<<
db_user=''           # DB User
db_dbname=''         # DB Schema
db_password=''       # DB Password
db_hostaddr=''       # DB Host address

dbconn = PostgresDBConn(
    port=5588,             # PostgreSQL port
    user=db_user,          # PostgreSQL username
    dbname=db_dbname,      # PostgreSQL database
    host=db_hostaddr,      # PostgreSQL address
    password=db_password,  # PostgreSQL password
)


In [ ]:
xmin, ymin, xmax, ymax = -70, 45, -58, 53
gulf_bbox = [xmin, xmax, ymin, ymax]
start_time = datetime(2023,8 , 10)
end_time = datetime(2023, 10, 9)

qry = DBQuery(
    dbconn=dbconn,
    start=start_time, end=end_time,
    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax,
    callback=aisdb.database.sqlfcn_callbacks.in_time_bbox_validmmsi
)

ais_tracks = []
rowgen = qry.gen_qry()
tracks = aisdb.track_gen.TrackGen(rowgen, decimate="True")

In [ ]:
#old method
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels', 
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': '10m_u_component_of_wind', 
        'area': [53, -70, 45, -58],
        'year': '2023',
        'month': '08',
        'day': '01',
        'time': '00:00',
    },
    'climate_data.grib')

In [ ]:
#new method
from aisdb.weather.weather_fetch import ClimateDataStore # for weather

climateDataStore = ClimateDataStore(dataset="reanalysis-era5-single-levels", short_names= ['10v','10u'], start_time = start_time, end_time = end_time, area= gulf_bbox)
climateDataStore.download_grib_file(output_folder="/weather_data")
